# 🧬 EHR → CRF Mapping Demo (Advanced Synthetic Pipeline)

This notebook demonstrates a synthetic example of mapping EHR-like data into CRF structures using:

✔ CDASH-style variables  
✔ JSON-based configuration mapping  
✔ Fuzzy matching to handle inconsistent field names  
✔ Unit normalization  
✔ Metadata assignment (attribute, LOINC code)  

⚠ **Note:** Everything here is synthetic and safe — no company or proprietary data.


## 🔧 Step 1 — Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## 📂 Step 2 — Load Synthetic EHR Dataset

In [ ]:
ehr = pd.read_csv('../data/ehr_synthetic.csv')
ehr.head()

## 📂 Step 3 — Load Mapping Configuration (JSON)

In [ ]:
with open('../config/mapping.json') as f:
    mapping = json.load(f)

mapping

## 🤖 Step 4 — Helper Functions: Fuzzy Matching + Unit Normalization

In [ ]:
def fuzzy_match(field, candidates, threshold=80):
    best = None
    best_score = 0
    for c in candidates:
        score = fuzz.ratio(field.lower(), c.lower())
        if score > best_score:
            best = c
            best_score = score
    return best if best_score >= threshold else None

def normalize_unit(value, from_unit, to_unit):
    if from_unit == to_unit:
        return value

    # Example conversions
    if from_unit == 'cm' and to_unit == 'm':
        return value / 100

    if from_unit == 'kg' and to_unit == 'g':
        return value * 1000

    return value  # default passthrough

## 🔄 Step 5 — Mapping Function (Core Logic)

In [ ]:
def apply_mapping(ehr_df, mapping):
    mapped_rows = []

    for _, row in ehr_df.iterrows():
        mapped = {}

        for ehr_field, m in mapping.items():

            # Fuzzy matching option
            candidates = m.get('keywords', [])
            _ = fuzzy_match(ehr_field, candidates) if candidates else None

            value = row.get(ehr_field, np.nan)

            # Normalize units if applicable
            unit = m.get('unit', None)
            if unit and not pd.isna(value):
                value = normalize_unit(value, unit, unit)

            # Assign mapped value
            mapped[m['cdash_variable']] = value

            # Metadata
            mapped[f"{m['cdash_variable']}_attr"] = m.get('attribute', 'VALUE')
            mapped[f"{m['cdash_variable']}_code"] = m.get('loinc_code', None)

        mapped_rows.append(mapped)

    return pd.DataFrame(mapped_rows)

## 🧪 Step 6 — Apply Mapping

In [ ]:
crf = apply_mapping(ehr, mapping)
crf.head()

## 📊 Step 7 — Visualize Before/After Mapping

In [ ]:
# Compare a few mapped fields visually
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(ehr['hemoglobin'], kde=True, ax=axs[0])
axs[0].set_title('Raw Hemoglobin Distribution')

sns.histplot(crf['LBORRES'], kde=True, ax=axs[1])
axs[1].set_title('Mapped LBORRES (Hemoglobin)')

plt.tight_layout()

## 💾 Step 8 — Save Output CRF Dataset

In [ ]:
crf.to_csv('../data/output_crf_dataset.csv', index=False)
print('CRF dataset saved to data/output_crf_dataset.csv')

# 🎉 Completed!

You now have an **advanced synthetic EHR→CRF mapping pipeline** demonstrating:

- JSON-based configuration mapping  
- CDASH-style variable attribution  
- LOINC-code metadata  
- Fuzzy matching  
- Unit normalization  
- CRF dataset generation  
- Before/after visualizations  

This is *exactly* the kind of project that impresses healthcare data science teams while keeping you fully NDA-safe.
